# Informe CEV (v.2) - Page 2

### Import `pdfplumber`

In [1]:
import pandas as pd
import fitz  # PyMuPDF

/home/roruizf/anaconda3/envs/data_env/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/roruizf/anaconda3/envs/data_env/lib/python3.9/site-packages/pandas/core/arrays/masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


### Load the PDF

In [2]:
pdf_file_path = '/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-database-reports/data/raw/9/pdf_files/9_195_1_aa1400a4dbe6c3177ff044edce8b5585fb846445.pdf'
pdf_file_path = './1_5_1_c9cbda772da622dd1667027eef1b862a7d7209c8.pdf'
pdf_file_path = './1_113_2_0da232073435d32084842b8fd244ecdf37be7c25.pdf'
#pdf_file_path = './Informe_CEV_v2.pdf'
pdf_report = fitz.open(pdf_file_path)
page_number = 1  # Page number (starting from 0)
page = pdf_report[page_number]

## Pagina 2

In [3]:
def extract_text_from_area(page, area):
    """
    Extract text from a specific area of a PDF page.

    Args:
    - page (fitz.Page): Page object from which to extract text.
    - area (tuple): Tuple containing (x1, y1, x2, y2) coordinates of the area to extract text from.

    Returns:
    - extracted_text (str): Text extracted from the specified area.
    """
    extracted_text = ""
    try:
        # Clean the page contents to avoid misplaced item insertions
        page.clean_contents()

        # Pdf Report Dimensions
        report_width = 215.9  # mm
        report_height = 330.0  # mm

        # Get page dimensions
        width = page.rect.width
        height = page.rect.height

        # Normalize the coordinates
        x1, y1, x2, y2 = area
        rx1, ry1, rx2, ry2 = x1 / report_width, y1 / report_height, x2 / report_width, y2 / report_height

        # Define the rectangle area to extract text from
        rect = fitz.Rect(rx1 * width, ry1 * height, rx2 * width, ry2 * height)

        # Extract text from the specified area
        extracted_text = page.get_textbox(rect)
    except Exception as e:
        print(f"Error: {e}")

    return extracted_text

### Seccion 1: Datos vivienda y Evaluación

#### Subsection 1: Izquierda

In [4]:
area_coordinates = (7.8, 46.3, 96.8, 74.2)  # Coordinates of the area to extract text from: (x1, y1, x2, y2)
extracted_text = extract_text_from_area(page, area_coordinates)
datos_vivienda = extracted_text.splitlines()[-5:]
# Swap the elements
datos_vivienda[2], datos_vivienda[3] = datos_vivienda[3], datos_vivienda[2]
datos_vivienda

['Región de Tarapacá',
 'Iquique',
 'RIQUELME N°931, BLOCK D, DEPTO ',
 '730-0010',
 'Departamento, Tipologia_Planta Amplia']

In [5]:
index = ['region', 'comuna', 'direccion', 'rol_vivienda', 'tipo_vivienda']

In [6]:
_dict = dict(zip(index, datos_vivienda))
_dict

{'region': 'Región de Tarapacá',
 'comuna': 'Iquique',
 'direccion': 'RIQUELME N°931, BLOCK D, DEPTO ',
 'rol_vivienda': '730-0010',
 'tipo_vivienda': 'Departamento, Tipologia_Planta Amplia'}

In [7]:
# Convert dictionary to DataFrame
df1 = pd.DataFrame.from_dict(_dict, orient='index').T
df1.T

,0
region,Región de Tarapacá
comuna,Iquique
direccion,"RIQUELME N°931, BLOCK D, DEPTO"
rol_vivienda,730-0010
tipo_vivienda,"Departamento, Tipologia_Planta Amplia"


#### Subsection 2: Derecha

In [8]:
area_coordinates = (98.6, 46.3, 209.3, 74.2)  # Coordinates of the area to extract text from: (x1, y1, x2, y2)
extracted_text = extract_text_from_area(page, area_coordinates)
datos_vivienda = extracted_text.splitlines()[-5:]
# Swap the elements
datos_vivienda[2], datos_vivienda[3] = datos_vivienda[3], datos_vivienda[2]
datos_vivienda

['A',
 '49,3',
 'SERVIU TARAPACÁ',
 'XIMENA ALEJANDRA AHUMADA OSSIO',
 '630a6f2020']

In [9]:
index = ['zona_termica', 'superficie_interior_util_m2', 'solicitado_por', 'evaluado_por', 'codigo_evaluacion']

In [10]:
_dict = dict(zip(index, datos_vivienda))
_dict

{'zona_termica': 'A',
 'superficie_interior_util_m2': '49,3',
 'solicitado_por': 'SERVIU TARAPACÁ',
 'evaluado_por': 'XIMENA ALEJANDRA AHUMADA OSSIO',
 'codigo_evaluacion': '630a6f2020'}

In [11]:
_dict['superficie_interior_util_m2'] = float(_dict['superficie_interior_util_m2'].replace(',', '.'))

In [12]:
# Convert dictionary to DataFrame
df2 = pd.DataFrame.from_dict(_dict, orient='index').T
df2.T

,0
zona_termica,A
superficie_interior_util_m2,49.3
solicitado_por,SERVIU TARAPACÁ
evaluado_por,XIMENA ALEJANDRA AHUMADA OSSIO
codigo_evaluacion,630a6f2020


In [13]:
df = pd.concat([df1, df2], axis=1)
df.T

,0
region,Región de Tarapacá
comuna,Iquique
direccion,"RIQUELME N°931, BLOCK D, DEPTO"
rol_vivienda,730-0010
tipo_vivienda,"Departamento, Tipologia_Planta Amplia"
zona_termica,A
superficie_interior_util_m2,49.3
solicitado_por,SERVIU TARAPACÁ
evaluado_por,XIMENA ALEJANDRA AHUMADA OSSIO
codigo_evaluacion,630a6f2020


### Seccion 2: Demanda energética promedio según tipología y zona térmica (kWh/m2 año)

#### Subsection: Demanda Energetica para Calefaccion

In [14]:
area_coordinates = (99.1, 99.1, 135.3, 105.1)  # Coordinates of the area to extract text from: (x1, y1, x2, y2)
extracted_text = extract_text_from_area(page, area_coordinates)
demanda_calefaccion_kwh_m2_ano = extracted_text.split('\n')[-1]
df['demanda_calefaccion_kwh_m2_ano'] = float(demanda_calefaccion_kwh_m2_ano.replace(',', '.'))
df.T

,0
region,Región de Tarapacá
comuna,Iquique
direccion,"RIQUELME N°931, BLOCK D, DEPTO"
rol_vivienda,730-0010
tipo_vivienda,"Departamento, Tipologia_Planta Amplia"
zona_termica,A
superficie_interior_util_m2,49.3
solicitado_por,SERVIU TARAPACÁ
evaluado_por,XIMENA ALEJANDRA AHUMADA OSSIO
codigo_evaluacion,630a6f2020


#### Subsection: Demanda Energetica para Enfriamiento

In [15]:
area_coordinates = (99.1, 120.5, 135.3, 126.5)  # Coordinates of the area to extract text from: (x1, y1, x2, y2)
extracted_text = extract_text_from_area(page, area_coordinates)
demanda_enfriamiento_kwh_m2_ano = extracted_text.split('\n')[-1]
df['demanda_enfriamiento_kwh_m2_ano'] = float(demanda_enfriamiento_kwh_m2_ano.replace(',', '.'))
df.T

,0
region,Región de Tarapacá
comuna,Iquique
direccion,"RIQUELME N°931, BLOCK D, DEPTO"
rol_vivienda,730-0010
tipo_vivienda,"Departamento, Tipologia_Planta Amplia"
zona_termica,A
superficie_interior_util_m2,49.3
solicitado_por,SERVIU TARAPACÁ
evaluado_por,XIMENA ALEJANDRA AHUMADA OSSIO
codigo_evaluacion,630a6f2020


#### Subsection: Demanda Energetica Total

In [16]:
area_coordinates = (99.1, 137.4, 135.3, 150.4)  # Coordinates of the area to extract text from: (x1, y1, x2, y2)
extracted_text = extract_text_from_area(page, area_coordinates)
demanda_total_kwh_m2_ano = extracted_text.split('\n')[-1]
df['demanda_total_kwh_m2_ano'] = float(demanda_total_kwh_m2_ano.replace(',', '.'))
df.T

,0
region,Región de Tarapacá
comuna,Iquique
direccion,"RIQUELME N°931, BLOCK D, DEPTO"
rol_vivienda,730-0010
tipo_vivienda,"Departamento, Tipologia_Planta Amplia"
zona_termica,A
superficie_interior_util_m2,49.3
solicitado_por,SERVIU TARAPACÁ
evaluado_por,XIMENA ALEJANDRA AHUMADA OSSIO
codigo_evaluacion,630a6f2020


### Seccion 3

### Subsection 1: Demanda Energética Total

In [17]:
area_coordinates = (39.2, 159.8, 122.8, 166.0)  # Coordinates of the area to extract text from: (x1, y1, x2, y2)
extracted_text = extract_text_from_area(page, area_coordinates)
demanda_total_bis_kwh_m2_ano = extracted_text.split('\n')[-1]
df['demanda_total_bis_kwh_m2_ano'] = float(demanda_total_bis_kwh_m2_ano.replace(',', '.'))
df.T

,0
region,Región de Tarapacá
comuna,Iquique
direccion,"RIQUELME N°931, BLOCK D, DEPTO"
rol_vivienda,730-0010
tipo_vivienda,"Departamento, Tipologia_Planta Amplia"
zona_termica,A
superficie_interior_util_m2,49.3
solicitado_por,SERVIU TARAPACÁ
evaluado_por,XIMENA ALEJANDRA AHUMADA OSSIO
codigo_evaluacion,630a6f2020


### Subsection 2: Demanda Energética Total de Referencia

In [18]:
area_coordinates = (16.9, 168.3, 146.2, 173.2)  # Coordinates of the area to extract text from: (x1, y1, x2, y2)
extracted_text = extract_text_from_area(page, area_coordinates)
demanda_total_referencia_kwh_m2_ano = extracted_text.split('\n')[-1]
df['demanda_total_referencia_kwh_m2_ano'] = float(demanda_total_referencia_kwh_m2_ano.replace(',', '.'))
df.T

,0
region,Región de Tarapacá
comuna,Iquique
direccion,"RIQUELME N°931, BLOCK D, DEPTO"
rol_vivienda,730-0010
tipo_vivienda,"Departamento, Tipologia_Planta Amplia"
zona_termica,A
superficie_interior_util_m2,49.3
solicitado_por,SERVIU TARAPACÁ
evaluado_por,XIMENA ALEJANDRA AHUMADA OSSIO
codigo_evaluacion,630a6f2020


### Porcentaje de Ahorro

In [19]:
area_coordinates = (152.0, 162.6, 201.5, 168.7)  # Coordinates of the area to extract text from: (x1, y1, x2, y2)
extracted_text = extract_text_from_area(page, area_coordinates)
porcentaje_ahorro = extracted_text.split('\n')[-1]
df['porcentaje_ahorro'] = float(porcentaje_ahorro.replace(',', '.'))
df.T

,0
region,Región de Tarapacá
comuna,Iquique
direccion,"RIQUELME N°931, BLOCK D, DEPTO"
rol_vivienda,730-0010
tipo_vivienda,"Departamento, Tipologia_Planta Amplia"
zona_termica,A
superficie_interior_util_m2,49.3
solicitado_por,SERVIU TARAPACÁ
evaluado_por,XIMENA ALEJANDRA AHUMADA OSSIO
codigo_evaluacion,630a6f2020


### Seccion 4: Principales características del Diseño de Arquitectura

### Muro Principal

In [20]:
# Muro principal: Descripcion
# Define section coordinates
area_coordinates = (46.2, 202.2, 184.5, 209.1)  # Coordinates of the area to extract text from: (x1, y1, x2, y2)
extracted_text = extract_text_from_area(page, area_coordinates)
muro_principal_descripcion = extracted_text.replace('\n', '')
df['muro_principal_descripcion'] = muro_principal_descripcion
df.T

,0
region,Región de Tarapacá
comuna,Iquique
direccion,"RIQUELME N°931, BLOCK D, DEPTO"
rol_vivienda,730-0010
tipo_vivienda,"Departamento, Tipologia_Planta Amplia"
zona_termica,A
superficie_interior_util_m2,49.3
solicitado_por,SERVIU TARAPACÁ
evaluado_por,XIMENA ALEJANDRA AHUMADA OSSIO
codigo_evaluacion,630a6f2020


In [21]:
# Muro principal: Exigencia
# Define section coordinates
area_coordinates = (185.5, 202.2, 209.5, 209.1)  # Coordinates of the area to extract text from: (x1, y1, x2, y2)
extracted_text = extract_text_from_area(page, area_coordinates)
muro_principal_exigencia = extracted_text.split('\n')[-1]
try:
    df['muro_principal_exigencia_W_m2_K'] = float(muro_principal_exigencia.replace('[W/m2K]', '').replace(',', '.'))
except ValueError:
    df['muro_principal_exigencia_W_m2_K'] = None
df.T

,0
region,Región de Tarapacá
comuna,Iquique
direccion,"RIQUELME N°931, BLOCK D, DEPTO"
rol_vivienda,730-0010
tipo_vivienda,"Departamento, Tipologia_Planta Amplia"
zona_termica,A
superficie_interior_util_m2,49.3
solicitado_por,SERVIU TARAPACÁ
evaluado_por,XIMENA ALEJANDRA AHUMADA OSSIO
codigo_evaluacion,630a6f2020


### Muro Secundario

In [22]:
# Muro secundario: Descripcion
# Define section coordinates
area_coordinates = (46.2, 209.2, 184.5, 215.1)  # Coordinates of the area to extract text from: (x1, y1, x2, y2)
extracted_text = extract_text_from_area(page, area_coordinates)
muro_secundario_descripcion = extracted_text.replace('\n', '')
df['muro_secundario_descripcion'] = muro_secundario_descripcion if muro_secundario_descripcion != '0' else None
df.T

,0
region,Región de Tarapacá
comuna,Iquique
direccion,"RIQUELME N°931, BLOCK D, DEPTO"
rol_vivienda,730-0010
tipo_vivienda,"Departamento, Tipologia_Planta Amplia"
zona_termica,A
superficie_interior_util_m2,49.3
solicitado_por,SERVIU TARAPACÁ
evaluado_por,XIMENA ALEJANDRA AHUMADA OSSIO
codigo_evaluacion,630a6f2020


In [23]:
# Muro secundario: Exigencia
# Define section coordinates
area_coordinates = (185.5, 202.2, 209.5, 209.1)  # Coordinates of the area to extract text from: (x1, y1, x2, y2)
extracted_text = extract_text_from_area(page, area_coordinates)
muro_secundario_exigencia = extracted_text.split('\n')[-1]
try:
    df['muro_secundario_exigencia_W_m2_K'] = float(muro_secundario_exigencia.replace('[W/m2K]', '').replace(',', '.'))
except ValueError:
    df['muro_secundario_exigencia_W_m2_K'] = None
df.T

,0
region,Región de Tarapacá
comuna,Iquique
direccion,"RIQUELME N°931, BLOCK D, DEPTO"
rol_vivienda,730-0010
tipo_vivienda,"Departamento, Tipologia_Planta Amplia"
zona_termica,A
superficie_interior_util_m2,49.3
solicitado_por,SERVIU TARAPACÁ
evaluado_por,XIMENA ALEJANDRA AHUMADA OSSIO
codigo_evaluacion,630a6f2020


### Piso Principal

In [24]:
# Piso principal: Descripcion
# Define section coordinates
area_coordinates = (46.2, 216.4, 184.5, 223.7)  # Coordinates of the area to extract text from: (x1, y1, x2, y2)
extracted_text = extract_text_from_area(page, area_coordinates)
piso_principal_descripcion = extracted_text.replace('\n', '')
df['piso_principal_descripcion'] = piso_principal_descripcion
df.T

,0
region,Región de Tarapacá
comuna,Iquique
direccion,"RIQUELME N°931, BLOCK D, DEPTO"
rol_vivienda,730-0010
tipo_vivienda,"Departamento, Tipologia_Planta Amplia"
zona_termica,A
superficie_interior_util_m2,49.3
solicitado_por,SERVIU TARAPACÁ
evaluado_por,XIMENA ALEJANDRA AHUMADA OSSIO
codigo_evaluacion,630a6f2020


In [25]:
# Piso principal: Exigencia
area_coordinates = (185.5, 216.4, 209.5, 223.7)  # Coordinates of the area to extract text from: (x1, y1, x2, y2)
extracted_text = extract_text_from_area(page, area_coordinates)
piso_principal_exigencia = extracted_text.split('\n')[-1]
try:
    df['piso_principal_exigencia_W_m2_K'] = float(piso_principal_exigencia.replace('[W/m2K]', '').replace(',', '.'))
except ValueError:
    df['piso_principal_exigencia_W_m2_K'] = None
df.T

,0
region,Región de Tarapacá
comuna,Iquique
direccion,"RIQUELME N°931, BLOCK D, DEPTO"
rol_vivienda,730-0010
tipo_vivienda,"Departamento, Tipologia_Planta Amplia"
zona_termica,A
superficie_interior_util_m2,49.3
solicitado_por,SERVIU TARAPACÁ
evaluado_por,XIMENA ALEJANDRA AHUMADA OSSIO
codigo_evaluacion,630a6f2020


### Puerta principal

In [26]:
# Puerta principal: Descripcion
area_coordinates = (46.2, 223.5, 184.5, 230.2)  # Coordinates of the area to extract text from: (x1, y1, x2, y2)
extracted_text = extract_text_from_area(page, area_coordinates)
puerta_principal_descripcion = extracted_text.replace('\n', '')
df['puerta_principal_descripcion'] = puerta_principal_descripcion
df.T

,0
region,Región de Tarapacá
comuna,Iquique
direccion,"RIQUELME N°931, BLOCK D, DEPTO"
rol_vivienda,730-0010
tipo_vivienda,"Departamento, Tipologia_Planta Amplia"
zona_termica,A
superficie_interior_util_m2,49.3
solicitado_por,SERVIU TARAPACÁ
evaluado_por,XIMENA ALEJANDRA AHUMADA OSSIO
codigo_evaluacion,630a6f2020


In [27]:
extracted_text

'Puerta PUERTA PRINCIPAL con U= 2,69 [W/m2K] y  0 % de vidrio'

In [28]:
# Puerta principal: Exigencia
# Define section coordinates
area_coordinates  = (185.5, 223.9, 209.5, 230.2)
extracted_text = extract_text_from_area(page, area_coordinates)
puerta_principal_exigencia = extracted_text.strip()
df['puerta_principal_exigencia_W_m2_K'] = puerta_principal_exigencia if puerta_principal_exigencia.isalpha() else None
df.T

,0
region,Región de Tarapacá
comuna,Iquique
direccion,"RIQUELME N°931, BLOCK D, DEPTO"
rol_vivienda,730-0010
tipo_vivienda,"Departamento, Tipologia_Planta Amplia"
zona_termica,A
superficie_interior_util_m2,49.3
solicitado_por,SERVIU TARAPACÁ
evaluado_por,XIMENA ALEJANDRA AHUMADA OSSIO
codigo_evaluacion,630a6f2020


### Techo Principal

In [29]:
# Techo principal: Descripcion
area_coordinates = (46.2, 230.5, 184.5, 237.0)  # Coordinates of the area to extract text from: (x1, y1, x2, y2)
extracted_text = extract_text_from_area(page, area_coordinates)
techo_principal_descripcion = extracted_text.replace('\n', '')
df['techo_principal_descripcion'] = techo_principal_descripcion
df.T

,0
region,Región de Tarapacá
comuna,Iquique
direccion,"RIQUELME N°931, BLOCK D, DEPTO"
rol_vivienda,730-0010
tipo_vivienda,"Departamento, Tipologia_Planta Amplia"
zona_termica,A
superficie_interior_util_m2,49.3
solicitado_por,SERVIU TARAPACÁ
evaluado_por,XIMENA ALEJANDRA AHUMADA OSSIO
codigo_evaluacion,630a6f2020


In [30]:
# Techo principal: Exigencia
# Define section coordinates
area_coordinates  = (185.5, 230.5, 209.5, 237.2)
extracted_text = extract_text_from_area(page, area_coordinates)
techo_principal_exigencia = extracted_text.strip()
try:
    df['techo_principal_exigencia_W_m2_K'] = float(techo_principal_exigencia.replace('[W/m2K]', '').replace(',', '.'))
except ValueError:
    df['techo_principal_exigencia_W_m2_K'] = None
df.T

,0
region,Región de Tarapacá
comuna,Iquique
direccion,"RIQUELME N°931, BLOCK D, DEPTO"
rol_vivienda,730-0010
tipo_vivienda,"Departamento, Tipologia_Planta Amplia"
zona_termica,A
superficie_interior_util_m2,49.3
solicitado_por,SERVIU TARAPACÁ
evaluado_por,XIMENA ALEJANDRA AHUMADA OSSIO
codigo_evaluacion,630a6f2020


### Techo Secundario

In [31]:
# Techo secundario: Descripcion
area_coordinates = (46.2, 237.2, 184.5, 244.1)  # Coordinates of the area to extract text from: (x1, y1, x2, y2)
extracted_text = extract_text_from_area(page, area_coordinates)
techo_secundario_descripcion = extracted_text.replace('\n', '')
df['techo_secundario_descripcion'] = techo_secundario_descripcion if techo_secundario_descripcion != '0' else None
df.T

,0
region,Región de Tarapacá
comuna,Iquique
direccion,"RIQUELME N°931, BLOCK D, DEPTO"
rol_vivienda,730-0010
tipo_vivienda,"Departamento, Tipologia_Planta Amplia"
zona_termica,A
superficie_interior_util_m2,49.3
solicitado_por,SERVIU TARAPACÁ
evaluado_por,XIMENA ALEJANDRA AHUMADA OSSIO
codigo_evaluacion,630a6f2020


In [32]:
# Techo secundario: Exigencia
# Define section coordinates
area_coordinates  = (185.5, 237.2, 209.5, 244.1)
extracted_text = extract_text_from_area(page, area_coordinates)
techo_secundario_exigencia = extracted_text.strip()
try:
    df['techo_secundario_exigencia_W_m2_K'] = float(techo_secundario_exigencia.replace('[W/m2K]', '').replace(',', '.'))
except ValueError:
    df['techo_principal_exigencia_W_m2_K'] = None

df.T

,0
region,Región de Tarapacá
comuna,Iquique
direccion,"RIQUELME N°931, BLOCK D, DEPTO"
rol_vivienda,730-0010
tipo_vivienda,"Departamento, Tipologia_Planta Amplia"
zona_termica,A
superficie_interior_util_m2,49.3
solicitado_por,SERVIU TARAPACÁ
evaluado_por,XIMENA ALEJANDRA AHUMADA OSSIO
codigo_evaluacion,630a6f2020


### Superficie vidriada principal

In [33]:
# Superficie vidriada principal: Descripcion
area_coordinates = (46.2, 244.2, 184.5, 251.0)  # Coordinates of the area to extract text from: (x1, y1, x2, y2)
extracted_text = extract_text_from_area(page, area_coordinates)
superficie_vidriada_principal_descripcion = extracted_text.replace('\n', '')
df['superficie_vidriada_principal_descripcion'] = superficie_vidriada_principal_descripcion
df.T

,0
region,Región de Tarapacá
comuna,Iquique
direccion,"RIQUELME N°931, BLOCK D, DEPTO"
rol_vivienda,730-0010
tipo_vivienda,"Departamento, Tipologia_Planta Amplia"
zona_termica,A
superficie_interior_util_m2,49.3
solicitado_por,SERVIU TARAPACÁ
evaluado_por,XIMENA ALEJANDRA AHUMADA OSSIO
codigo_evaluacion,630a6f2020


In [34]:
# Superficie vidriada principal: Exigencia
# Define section coordinates
area_coordinates  = (185.5, 244.3, 209.5, 251.0)
extracted_text = extract_text_from_area(page, area_coordinates)
superficie_vidriada_principal_exigencia = extracted_text.strip()
df['superficie_vidriada_principal_exigencia'] = superficie_vidriada_principal_exigencia.strip() if superficie_vidriada_principal_exigencia.isalpha() else None
df.T

,0
region,Región de Tarapacá
comuna,Iquique
direccion,"RIQUELME N°931, BLOCK D, DEPTO"
rol_vivienda,730-0010
tipo_vivienda,"Departamento, Tipologia_Planta Amplia"
zona_termica,A
superficie_interior_util_m2,49.3
solicitado_por,SERVIU TARAPACÁ
evaluado_por,XIMENA ALEJANDRA AHUMADA OSSIO
codigo_evaluacion,630a6f2020


### Superficie vidriada secundaria

In [35]:
# Superficie vidriada secundaria: Descripcion
area_coordinates = (46.2, 251.3, 184.5, 258.0)  # Coordinates of the area to extract text from: (x1, y1, x2, y2)
extracted_text = extract_text_from_area(page, area_coordinates)
superficie_vidriada_secundaria_descripcion = extracted_text.replace('\n', '')
df['superficie_vidriada_secundaria_descripcion'] = superficie_vidriada_secundaria_descripcion if superficie_vidriada_secundaria_descripcion != '0' else None
df.T

,0
region,Región de Tarapacá
comuna,Iquique
direccion,"RIQUELME N°931, BLOCK D, DEPTO"
rol_vivienda,730-0010
tipo_vivienda,"Departamento, Tipologia_Planta Amplia"
zona_termica,A
superficie_interior_util_m2,49.3
solicitado_por,SERVIU TARAPACÁ
evaluado_por,XIMENA ALEJANDRA AHUMADA OSSIO
codigo_evaluacion,630a6f2020


In [36]:
# Superficie vidriada secundaria: Exigencia
# Define section coordinates
area_coordinates  = (185.5, 251.3, 209.5, 258.0)
extracted_text = extract_text_from_area(page, area_coordinates)
superficie_vidriada_secundaria_exigencia = extracted_text.strip()
df['superficie_vidriada_secundaria_exigencia'] = superficie_vidriada_secundaria_exigencia.strip() if superficie_vidriada_secundaria_exigencia.isalpha() else None
df.T

,0
region,Región de Tarapacá
comuna,Iquique
direccion,"RIQUELME N°931, BLOCK D, DEPTO"
rol_vivienda,730-0010
tipo_vivienda,"Departamento, Tipologia_Planta Amplia"
zona_termica,A
superficie_interior_util_m2,49.3
solicitado_por,SERVIU TARAPACÁ
evaluado_por,XIMENA ALEJANDRA AHUMADA OSSIO
codigo_evaluacion,630a6f2020


### Ventilación (RAH)

In [37]:
# Ventilación (RAH): Descripcion
area_coordinates = (46.2, 258.3, 184.5, 265.0)
extracted_text = extract_text_from_area(page, area_coordinates)
ventilacion_rah_descripcion = extracted_text
df['ventilacion_rah_descripcion'] = ventilacion_rah_descripcion.replace('\n', '').strip()
df.T

,0
region,Región de Tarapacá
comuna,Iquique
direccion,"RIQUELME N°931, BLOCK D, DEPTO"
rol_vivienda,730-0010
tipo_vivienda,"Departamento, Tipologia_Planta Amplia"
zona_termica,A
superficie_interior_util_m2,49.3
solicitado_por,SERVIU TARAPACÁ
evaluado_por,XIMENA ALEJANDRA AHUMADA OSSIO
codigo_evaluacion,630a6f2020


In [38]:
# Ventilación (RAH): Exigencia
# Define section coordinates
area_coordinates = (185.5, 258.3, 209.5, 265.0)
extracted_text = extract_text_from_area(page, area_coordinates)
ventilacion_rah_exigencia = extracted_text
df['ventilacion_rah_exigencia'] = ventilacion_rah_exigencia.strip()
df.T

,0
region,Región de Tarapacá
comuna,Iquique
direccion,"RIQUELME N°931, BLOCK D, DEPTO"
rol_vivienda,730-0010
tipo_vivienda,"Departamento, Tipologia_Planta Amplia"
zona_termica,A
superficie_interior_util_m2,49.3
solicitado_por,SERVIU TARAPACÁ
evaluado_por,XIMENA ALEJANDRA AHUMADA OSSIO
codigo_evaluacion,630a6f2020


### Inﬁltraciones (RAH)

In [39]:
# Infiltraciones (RAH): Descripcion
area_coordinates = (46.2, 265.3, 184.5, 272.0)
extracted_text = extract_text_from_area(page, area_coordinates)
infiltraciones_rah_descripcion = extracted_text
df['infiltraciones_rah_descripcion'] = infiltraciones_rah_descripcion.replace('\n', '').strip()
df.T

,0
region,Región de Tarapacá
comuna,Iquique
direccion,"RIQUELME N°931, BLOCK D, DEPTO"
rol_vivienda,730-0010
tipo_vivienda,"Departamento, Tipologia_Planta Amplia"
zona_termica,A
superficie_interior_util_m2,49.3
solicitado_por,SERVIU TARAPACÁ
evaluado_por,XIMENA ALEJANDRA AHUMADA OSSIO
codigo_evaluacion,630a6f2020


In [40]:
# Infiltraciones (RAH): Exigencia
area_coordinates = (185.5, 265.3, 209.5, 272.0)
extracted_text = extract_text_from_area(page, area_coordinates)
infiltraciones_rah_exigencia = extracted_text
df['infiltraciones_rah_exigencia'] = infiltraciones_rah_exigencia.replace('\n', '').strip()
df.T

,0
region,Región de Tarapacá
comuna,Iquique
direccion,"RIQUELME N°931, BLOCK D, DEPTO"
rol_vivienda,730-0010
tipo_vivienda,"Departamento, Tipologia_Planta Amplia"
zona_termica,A
superficie_interior_util_m2,49.3
solicitado_por,SERVIU TARAPACÁ
evaluado_por,XIMENA ALEJANDRA AHUMADA OSSIO
codigo_evaluacion,630a6f2020


### END